In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import geopandas as gpd
import seaborn as sns
import json
import random
import matplotlib.pylab as plt
%matplotlib inline

# MCLP를 위한 선형계획법 툴
from pulp import *

# 시각화 툴 : Pydeck
import pydeck as pdk

# 지리 데이터 전처리 툴 : Shapely
import shapely.speedups
shapely.speedups.enable()
from shapely.ops import unary_union
from shapely.geometry import Point, MultiLineString, mapping, shape

mapbox_key = 'pk.eyJ1IjoicWlxaTY1NCIsImEiOiJja2xnYW0xdWgyMmUyMnVxZWl2NGJpYng3In0.VIYWYBixklc-pAM9w7AjMA'

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

<ipython-input-1-9e27b1ced98f>:19: DeprecationWarning: This function has no longer any effect, and will be removed in a future release. Starting with Shapely 2.0, equivalent speedups are always available
  shapely.speedups.enable()


In [2]:
tile = gpd.read_file("C:/Users/82104/Documents/의료최종_20230522.shp",encoding='euc-kr')
phar = pd.read_csv("C:/Users/82104/Documents/카카오톡 받은 파일/약국매출_위도경도.csv",encoding='euc-kr')
hos = pd.read_csv("C:/Users/82104/전종설/응급실 위치.csv", encoding='euc-kr')
real_phar = pd.read_csv("C:/Users/82104/Downloads/공공심야약국_위도경도.csv")

In [3]:
real_phar.head()

,연번,자치구,약국명,주소,Latitude,Longitude,전화번호,운영시간
0,1,종로구,일등약국,"서울특별시 종로구 경교장길 35, 상가동 3148,3149호(평동, 경희궁자이3단지)",37.568718,126.966158,02-723-7581,365일
1,2,중구,대풍약국,서울특별시 중구 동호로11길 43 (신당동),37.553336,127.009063,02-2252-3944,"월,토,일"
2,3,용산구,수복약국,서울특별시 용산구 새창로12길 12 (도원동),37.539267,126.957519,02-714-0667,365일
3,4,성동구,새인선약국,서울특별시 성동구 광나루로11길 41-1,37.551494,127.067860,02-466-4159,365일
4,5,광진구,더클래식약국,"서울특별시 광진구 능동로 90, 1층 104-2호",37.538788,127.071137,02-455-4565,365일


In [4]:
phar.isnull().sum()

Unnamed: 0       0
개방자치단체코드         0
관리번호             0
인허가일자            0
인허가취소일자       5431
영업상태코드           0
영업상태명            0
상세영업상태코드         0
상세영업상태명          0
폐업일자          5422
휴업시작일자        5430
휴업종료일자        5430
재개업일자         5439
전화번호           208
소재지면적         5439
소재지우편번호       3475
지번주소           307
도로명주소            1
도로명우편번호         51
사업장명             0
최종수정일자           0
데이터갱신구분          0
데이터갱신일자          0
업태구분명         5439
좌표정보(X)          0
좌표정보(Y)          0
약국영업면적         707
지정일자           702
좌표정보(X).1        0
좌표정보(Y).1        0
경도               0
위도               0
dtype: int64

In [5]:
phar=phar.dropna(subset=['도로명주소'], how='any', axis=0)

#phar['도로명주소'].dropna(axis=0)

In [6]:
phar.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5438 entries, 0 to 5438
Data columns (total 32 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5438 non-null   int64  
 1   개방자치단체코드    5438 non-null   int64  
 2   관리번호        5438 non-null   object 
 3   인허가일자       5438 non-null   object 
 4   인허가취소일자     8 non-null      float64
 5   영업상태코드      5438 non-null   int64  
 6   영업상태명       5438 non-null   object 
 7   상세영업상태코드    5438 non-null   int64  
 8   상세영업상태명     5438 non-null   object 
 9   폐업일자        16 non-null     object 
 10  휴업시작일자      9 non-null      object 
 11  휴업종료일자      9 non-null      object 
 12  재개업일자       0 non-null      float64
 13  전화번호        5230 non-null   object 
 14  소재지면적       0 non-null      float64
 15  소재지우편번호     1963 non-null   object 
 16  지번주소        5131 non-null   object 
 17  도로명주소       5438 non-null   object 
 18  도로명우편번호     5388 non-null   float64
 19  사업장명        5438 non-null   obje

In [7]:
phar2=phar[['도로명주소','위도','경도']]

In [8]:
phar2 = gpd.GeoDataFrame(phar2, geometry=gpd.points_from_xy(phar2.경도, phar2.위도))

In [9]:
phar2 = phar2.reset_index()
phar2.head()

,index,도로명주소,위도,경도,geometry
0,0,"서울특별시 동대문구 경희대로 17, 1,2층 (회기동)",37.593225,127.049584,POINT (127.04958 37.59323)
1,1,"서울특별시 강남구 강남대로 606, 삼주빌딩 1층 일부호 (논현동)",37.515862,127.017298,POINT (127.01730 37.51586)
2,2,"서울특별시 서대문구 충정로 70, 웨스트게이트타워 2층 (미근동)",37.564824,126.963507,POINT (126.96351 37.56482)
3,3,"서울특별시 종로구 종로 22, 606호 (서린동, 인주빌딩)",37.569887,126.976742,POINT (126.97674 37.56989)
4,4,"서울특별시 광진구 아차산로 361, 1층 (자양동)",37.537028,127.080037,POINT (127.08004 37.53703)


In [10]:
tile2=tile[['id','adm_nm','final','lon','lat']]
tile2 = gpd.GeoDataFrame(tile2, geometry=gpd.points_from_xy(tile2.lon, tile.lat))
tile2 = tile2.reset_index()
tile2.head()

,index,id,adm_nm,final,lon,lat,geometry
0,0,11135.0,"서울특별시 종로구 사직동,서울특별시 종로구 교남동,서울특별시 중구 소공동,서울특별시...",24235,126.966778,37.56858,POINT (126.96678 37.56858)
1,1,10850.0,"서울특별시 종로구 사직동,서울특별시 종로구 무악동,서울특별시 종로구 교남동,서울특별...",13640,126.961778,37.57858,POINT (126.96178 37.57858)
2,2,4450.0,"서울특별시 종로구 사직동,서울특별시 종로구 청운효자동",1709,126.966778,37.57858,POINT (126.96678 37.57858)
3,3,8684.0,"서울특별시 종로구 사직동,서울특별시 종로구 무악동,서울특별시 종로구 교남동,서울특별...",41909,126.961778,37.57358,POINT (126.96178 37.57358)
4,4,7008.0,"서울특별시 종로구 사직동,서울특별시 종로구 종로1·2·3·4가동,서울특별시 종로구 ...",18303,126.976778,37.57358,POINT (126.97678 37.57358)


In [11]:
tile2.crs = 'epsg:4326'
phar2.crs = 'epsg:4326'

tile2 = tile2.to_crs(epsg=5179) 
phar2 = phar2.to_crs(epsg=5179)

In [ ]:
# 각 약국과 기준 좌표들 간의 거리 행렬
d = np.empty((len(tile2), len(phar2)))
for i in range(len(tile2)):
    point_i = tile2.geometry[i]
    d[i] = [point_i.distance(point_j) for point_j in phar2.geometry]
    if i % 2000 == 0:
        print('loading'+ '.'*(i//2000)+ '_'*(6-i//2000))
d.shape

loading______


In [ ]:
# 약국사이의 거리
# 약국 간에 최소 거리가 어느 정도 이상이어야 한다는 제약조건을 반영하기 위함.
r = np.empty((len(phar2), len(phar2)))
for i in range(len(phar2)):
    phar2_i = phar2.geometry[i]
    r[i] = [ phar2_i.distance(phar2_j) for phar2_j in phar2.geometry]
r.shape

In [ ]:
I = tile2.index.values   #타일(수요지)
J = phar2.index.values   #약국후보(시설입지)
S = 1600                  # 서비스 영역 반경
min_dist = 3000             #약국과 약국 사이의 최소 거리

a = tile2.final.values
P = 36                 # 선정할 심야약국 개수

# Compute the sets Ni
# NB: this will be a list in which each item is a list of nodes
# within the threshold distance of the i'th node
N = [[j for j in J if d[i][j] < S] for i in I]
R = [[int(r[phar2_i][phar2_j] < min_dist) for phar2_j in J ] for phar2_i in J]
# Formulate optimisation

prob = LpProblem("MCLP", LpMaximize)
x = LpVariable.dicts("x", J, lowBound=0, upBound=1, cat='Integer')
y = LpVariable.dicts("y", I, lowBound=0, upBound=1, cat='Integer')

# Objective
prob += lpSum([a[i]*y[i] for i in I])

# Constraints
for i in I:
    prob += lpSum([x[j] for j in N[i]]) >= y[i]
for j in J:
    prob += lpSum([x[rr] for rr in R[j]]) <= 1
    

prob += lpSum([x[j] for j in J]) == P

# Solve problem
prob.solve()

x_soln = np.array([x[j].varValue for j in J])

# And print some output
print (("Night Pharmacy:"), LpStatus[prob.status])
print ("Weight Covered is = ", value(prob.objective))
print ("x = ", x_soln)

In [ ]:
phar2['opt'] = x_soln
phar2.head()

In [ ]:
opt_id = phar2[phar2['opt'] == 1]['도로명주소'].values
# 원래 데이터에서 후보지로 선택 필터링
phar['optimal'] = phar['도로명주소'].apply(lambda x: x in opt_id)
phar[phar['optimal'] == True]

In [ ]:
seoul_hang = gpd.read_file("C:/Users/82104/Documents/20230523_서울시경계.geojson")
seoul_hang['coordinates'] = seoul_hang['geometry'].apply(lambda x : mapping(x)['coordinates'][0])

# 서울 중심, 시각화에 사용될 예정
center = [37.5642135, 127.0016985]  
seoul_hang.head()

In [ ]:
base = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    seoul_hang, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255, 255, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.005
)

In [ ]:
scat1 = pdk.Layer('ScatterplotLayer',
                  phar[phar['optimal']==0], 
                  get_position = ['경도','위도'], 
                  get_fill_color='[0,255,0]',
                  get_radius=50,
                  opacity = 0.1,
                  pickable=True, auto_highlight=True
                 ) 

scat2 = pdk.Layer('ScatterplotLayer',
                  phar[phar['optimal']==1], 
                  get_position = ['경도','위도'], 
                  get_fill_color='[127,255,212,100]',
                  get_radius=1600,
                  pickable=True, auto_highlight=True 
                 ) 


scat3 = pdk.Layer('ScatterplotLayer',
                  hos,
                  get_position = ['병원경도','병원위도'],
                  get_fill_color = '[255,0,0]',
                  get_radius = 200, #반경
                  pickable=True, auto_highlight=True
                 )

scat4 = pdk.Layer('ScatterplotLayer',
                  real_phar,
                  get_position = ['Longitude','Latitude'],
                  get_fill_color = '[0,0,255,100]',
                  get_radius = 1600,
                  pickable=True, auto_highlight=True
                 )


center = center
view_state = pdk.ViewState( 
    longitude=center[1], 
    latitude=center[0], 
    zoom=11
) 

base = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    base, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[255, 255, 255]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    opacity = 0.005
)



r = pdk.Deck(layers=[base, scat1, scat2,scat3,scat4], initial_view_state=view_state,
            #mapbox_key = "pk.eyJ1IjoiamphY2tqamFja2VlIiwiYSI6ImNsaHpqNDJpNzFlbGYzb3Fhc2w3YmNna3UifQ.b2BZF1Zx6QqTufi5bdzUxg") 
            )
r.to_html()